<a href="https://colab.research.google.com/github/suuuuwimmer/study/blob/master/5-2.%20HyperParameterWithTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 검증 세트

**하이퍼 파라미터 튜닝**은 사용자가 여러 값들을 변경해가며 가장 좋은 성능을 뽑는 설정값을 결정하는 과정이다.\
테스트 세트로 여러번 성능 검사를 하면 모델이 조금씩 테스트 세트에 대해 훈련하게 되기 때문에 테스트 세트로 검사는 맨 마지막에 진행해야한다.\
그러므로 훈련 세트를 나눠 **검증 세트**를 만들어 검증 세트로 간이 성능 테스트를 하는 방법을 이용한다.

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

훈련 세트 sub_input, sub_target과 검증 세트 val_input, val_target으로 기존 훈련세트를 다시 나누는 과정

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

교차 검증이란 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복하는 것이다.\
또한 3-폴드 교차 검증이라는 것도 있는데, 이는 훈련 세트를 세 부분으로 나눠 교차 검증을 수행하는 것을 말한다.\
cf. k-폴드 교차 검증도 있음

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01341891, 0.02167416, 0.02525187, 0.04882073, 0.03598666]), 'score_time': array([0.0027864 , 0.0019815 , 0.00886154, 0.01437068, 0.02624893]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate()라는 메소드는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다.\
처음 두 개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다.\
각 키마다 5개의 숫자가 담겨있는데 기본적으로 5-폴드 교차 검증을 수행하기 때문이다.

In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


훈련 세트를 섞으려면 분활기를 지정해야한다.\
분활기는 교차 검증에서 폴드를 어떻게 나눌지를 결정한다.\
cross-validate()함수는 회귀에서 기본적으로 KFold 분활기를 사용하고 분류에서는 StratifiedKFold를 사용한다.

In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

모델이 학습하는 파라미터를 모델 파라미터라고 부르는데, 이때 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 한다.\
이런 하이퍼파라미터는 모두 클래스나 메소드의 매개변수로 사용된다.\
이런 매개변수들이 너무 많아질때 사이킷런의 **그리드 서치**를 사용할 수 있다.\
그리드 서치란 머신러닝에서 최적의 하이퍼파라미터 조합을 찾는 방법이다.\
모든 조합을 시도해보고, 가장 성능이 좋은 조합을 찾아낸다.\
매개변수와 탐색할 값의 리스트를 딕셔너리로 만드는 방법이 있다.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

이 그리드 서치에서는 결정 트리의 하이퍼파라미터인 min_inpurity_decrease 중 최적의 조합을 찾아내고자 하는 과정이다.\
min_inpurity_decrease란 노드의 불순도를 의미한다.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [ ]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야한다.\
사이킷런의 그리드 서치로 찾은 최적의 하이퍼파라미터를 가지고 자동으로 다시 모델이 훈련된다.

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_paramas_ 속성에 저장돼 있다.

In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 mean_test_score 키에 저장돼있다.

In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


교차 검증의 평균 점수를 일일이 확인해 가장 큰 점수를 고르는 것보다 넘파이의 argmax() 함수를 사용해 가장 큰 값의 인덱스를 추출할 수 있다.

그리드 서치를 이용해 최적의 하이퍼파라미터 조합을 찾는 과정\
1. 탐색할 매개변수를 고른다.
2. 훈련 세트에서 그리드 서치를 수행해 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾아 그리드 서치 객체에 저장한다.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

np.arange()는 넘피 라이브러리에 속해있으며 정수와 실수 모두 가능하다.\
그런데 range()는 파이썬 내장 함수로 정수만 사용 가능하고 연산이 불가능하다.

이때 교차 검증 횟수는 9*15*10으로 1350개이다. \
즉, 교차 검증으로 만들어지는 모델의 수는 6750개가 됨.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #그리드 서치 실행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

매개변수의 값이 수치일때 값의 범위나 간격을 정하기 어렵거나 너무 많아 서치 수행 시간이 오래 걸릴 수 있다. 이때 **랜덤 서치**가 필요한 것.\
랜덤 서치에는 매개변수 값의 목록을 전달한느 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다. \
이때 **사이파이**를 이용한다.

In [ ]:
from scipy.stats import uniform, randint

사이파이는 과학 라이브러리로 수치 계산에 용이한 라이브러리다.\
uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는 균등 분포 샘플링을 진행한다.\
randint는 정숫값, uniform은 실숫값을 뽑는다.

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 4, 3, 6, 3, 1, 1, 6, 1, 6])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([121,  99,  81,  98, 106,  85,  93,  94, 109, 114]))

10에서 10개를 추출했을때 아주 고르게 추출된 것을 볼 수 있다.

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.60857829, 0.2795936 , 0.4059522 , 0.47695652, 0.7427586 ,
       0.9801252 , 0.05012329, 0.79357074, 0.16195204, 0.33820475])

0과 1 사이에서 10개의 실수를 추출한 것

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e5225466d40>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e5225464b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e52254661d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e5225467c10>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e5225466d40>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e5225464b50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e52254661d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e5225467c10>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
